Meta, Apple and Tesla stock news are extracted using beautifulsoup

In [2]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup

In [11]:
limit = 3 # setting to 3 articles limit per page

def getDate(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    dates = soup.find_all(class_ = "date") 
    dateArray=[]
    count = 1
    for date in dates :
        dateArray.append(date.get_text().split()[1:])
        if count >= limit :
            break
        count += 1    
    return dateArray

# tweak start and end for faster compile
def getStartEnd(url,startnum,endnum):
    try :
        check = True
        start = getDate("{}{}".format(url,startnum))
        while check :
            if (start[2][0] == 'Aug' and start[2][2] == '2022') :
                check = False
                break
            startnum += 1
            start = getDate("{}{}".format(url,startnum))
            print(start)
        print(startnum)
        check = True
        end = getDate("{}{}".format(url,endnum))
        while check :
            if end[2][0] == 'Jan' and end[2][2] == '2018' :
                check = False
                break
            endnum += 1
            end = getDate("{}{}".format(url,endnum))    
            print(end)
        print(endnum)
        return (startnum,endnum)
    except : 
        print("error")

def getArticles(url,startnum,endnum):
    df = pd.DataFrame(columns=['Date','Title','Text'])
    start,end = getStartEnd(url,startnum,endnum)
    while start <= end :
        page = requests.get("{}{}".format(url,start))
        soup = BeautifulSoup(page.content, 'html.parser')
        dates = soup.find_all(class_="date")
        titles = soup.find_all(class_ = "title") 
        texts = soup.find_all("p")
        nTexts = []
        textCount = 0
        for t in texts :
            if t.text[:2] == "By":
                nTexts.append(t.text)
                textCount += 1
            if textCount == limit :
                break
        for i in range(textCount) :
            df_new = pd.DataFrame({"Date" : dates[i].text[3:], "Title" : titles[i].text, "Text" : nTexts[i]},index=[1])
            df = pd.concat([df,df_new],ignore_index=True)
        print(start, end, "[Page {}/{}]".format(start, end))
        start += 1
    return df

In [14]:
df_tsla = getArticles("https://www.investing.com/equities/tesla-motors-news/",36,811)

36
811
36 811 [Page 36/811]
37 811 [Page 37/811]
38 811 [Page 38/811]
39 811 [Page 39/811]
40 811 [Page 40/811]
41 811 [Page 41/811]
42 811 [Page 42/811]
43 811 [Page 43/811]
44 811 [Page 44/811]
45 811 [Page 45/811]
46 811 [Page 46/811]
47 811 [Page 47/811]
48 811 [Page 48/811]
49 811 [Page 49/811]
50 811 [Page 50/811]
51 811 [Page 51/811]


KeyboardInterrupt: 

In [17]:
df_aapl = getArticles("https://www.investing.com/equities/apple-computer-inc-news/",30,1034) 

30
[['Mar', '23,', '2018'], ['Mar', '23,', '2018'], ['Mar', '23,', '2018']]
[['Mar', '22,', '2018'], ['Mar', '22,', '2018'], ['Mar', '22,', '2018']]
[['Mar', '21,', '2018'], ['Mar', '21,', '2018'], ['Mar', '21,', '2018']]
[['Mar', '20,', '2018'], ['Mar', '20,', '2018'], ['Mar', '20,', '2018']]
[['Mar', '19,', '2018'], ['Mar', '19,', '2018'], ['Mar', '19,', '2018']]
[['Mar', '18,', '2018'], ['Mar', '18,', '2018'], ['Mar', '17,', '2018']]
[['Mar', '15,', '2018'], ['Mar', '15,', '2018'], ['Mar', '15,', '2018']]
[['Mar', '14,', '2018'], ['Mar', '14,', '2018'], ['Mar', '14,', '2018']]
[['Mar', '13,', '2018'], ['Mar', '13,', '2018'], ['Mar', '12,', '2018']]
[['Mar', '12,', '2018'], ['Mar', '12,', '2018'], ['Mar', '09,', '2018']]
[['Mar', '08,', '2018'], ['Mar', '08,', '2018'], ['Mar', '08,', '2018']]
[['Mar', '06,', '2018'], ['Mar', '05,', '2018'], ['Mar', '05,', '2018']]
[['Mar', '02,', '2018'], ['Mar', '02,', '2018'], ['Mar', '02,', '2018']]
[['Feb', '28,', '2018'], ['Feb', '28,', '2018'],

KeyboardInterrupt: 

In [21]:
df_meta = getArticles("https://www.investing.com/equities/facebook-inc-news/",20,1211) 

20
[['Feb', '14,', '2018'], ['Feb', '13,', '2018'], ['Feb', '13,', '2018']]
[['Feb', '13,', '2018'], ['Feb', '12,', '2018'], ['Feb', '12,', '2018']]
[['Feb', '10,', '2018'], ['Feb', '09,', '2018'], ['Feb', '09,', '2018']]
[['Feb', '09,', '2018'], ['Feb', '09,', '2018'], ['Feb', '09,', '2018']]
[['Feb', '08,', '2018'], ['Feb', '08,', '2018'], ['Feb', '08,', '2018']]
[['Feb', '06,', '2018'], ['Feb', '06,', '2018'], ['Feb', '06,', '2018']]
[['Feb', '05,', '2018'], ['Feb', '05,', '2018'], ['Feb', '05,', '2018']]
[['Feb', '03,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '02,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018']]
1211
20 1211 [Page 20/1211]


KeyboardInterrupt: 

In [6]:
df_tsla.to_csv('../Data/Stock/tsla_news.csv',index=False)
df_aapl.to_csv('../Data/Stock/aapl_news.csv',index=False)
df_meta.to_csv('../Data/Stock/meta_news.csv',index=False)